In [2]:
import pandas as pd
import argparse
import logging
import json
import os
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [3]:
# Define constants

BELLINZONA_UST = 2101
BLENIO_UST = 2102
LEVENTINA_UST = 2103
LOCARNO_UST = 2104
LUGANO_UST = 2105
MENDRISIO_UST = 2106
RIVIERA_UST = 2107
VALLEMAGGIA = 2108

### ODIS ### 
# AVG_RES_LIGHT_DENSITY = 8.3 # kWh/m2y ODIS
# AVG_RES_APP =  1400 # kWh/y ODIS, 

### ROHDATEN Tarife 2020 ELCOM ###
# H1 - 1600 kWh/an: logement de 2 pièces avec cuisinière électrique
# H2 - 2500 kWh/an: logement de 4 pièces avec cuisinière électrique; 
# H4 - 4500 kWh/an: logement de 5 pièces avec cuisinière électrique et sèche-linge (sans chauffe-eau électrique) 
# H8 - 7500 kWh/an: grand logement en propriété, avec large utilisation de l'électricité 
# C1 - 8000 kWh/an: très petite entreprise, puissance max.: 8 kW 
# C2 - 30 000 kWh/an: petite entreprise, puissance max.: 15 kW 
# C3 - 150 000 kWh/an: entreprise moyenne, puissance max.: 50 kW
# C4 - 500 000 kWh/an: grande entreprise, puissance max.: 150 kW, courant basse tension / C5 - courant moyenne tension, propre station de transformation 
# C6 - 1 500 000 kWh/an: grande entreprise, puissance max.: 400 kW, courant moyenne tension, propre station de transformation 
# C7 - 7 500 000 kWh/an: grande entreprise, puissance max.: 1630 kW, courant moyenne tension, propre station de transformation 

### Consumi di energia (in gigawattora), secondo la destinazione e il vettore energetico, in Ticino, nel 2020
S1_AGR_elec = 64000 # kWh/y
S2_IND_elec = 160000 #kWh/y
S3_SER_elec = 25000 #kWh/y

### https://pubdb.bfe.admin.ch/it/publication/download/10559 ###
MFH_2pp_elec = 2190 # kWh/y =  apps and light
MFH_1pp_elec_diff = 458.5 # kWh/y =  apps and light
SFH_4pp_elec = 4048 # kWh/y =  apps and light
SFH_1pp_elec_diff = 593.5 # kWh/y =  apps and light
# --> MFH 2PP = 2190
# --> SHF 3PP = 3454.5

AVG_DHW_PP = 40 # L/DAY/PERSON ODIS
AVG_M2_PP = 49.1 #TI - Superficie media per occupante, per Cantone, 2021, https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/superficie-persona.html
SUPERFICIE_MEDIA = 97.8 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/dimensioni.html

MFH_PP = 2.1 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/densita-utilizzazione.html
SFH_PP = 2.7

GKLAS = [1110,1121,1122,1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1241, 1242, 1271, 1274, 1275, 1276, 1277, 1278, 1231, 1252, 1273]

GKLAS_SFH = 1110
GKLAS_DFH = 1121
GKLAS_MFH = 1122
GKLAS_RES = [GKLAS_SFH, GKLAS_DFH, GKLAS_MFH]

GKLAS_HTG = [1110, 1121, 1122, 1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1231]
GKLAS_NOHTG = [1241, 1242, 1252, 1271, 1273, 1274, 1275, 1276, 1277, 1278]

GBAUP = [8011,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022,8023]
GSTAT = 1004 # existing
   
pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'

#SELECT CANTON
#CANTON = "Genève"
CANTON = "Ticino"

# Read the shp file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
SWISSTOPO_DISTRICT_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_BEZIRKSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html
SWISSTOPO_CANTON_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html. NOTE: The actual shapefile (.shp) is useless without the companion files: .dbf, .shx, .prj etc..
### SWISSTOPO_CANTON_FILE = figureout how to get the municipal boundaries https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill

CANTON_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_CANTON_FILE
DISTRICT_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_DISTRICT_FILE

CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

### AGR, IND, SER

In [3]:
# Calculate the electric demand for agriculture, industry, and services - top down - CODE WORKS

### Consumi di energia (in gigawattora), secondo la destinazione e il vettore energetico, in Ticino, nel 2020
S1_AGR_elec = 61640 # kWh/y = consumo "Altro (cantieri, agricoltura ecc.)" 2020 / Aziende primario 2020
S2_IND_elec = 153650 #kWh/y consumo "Artigianato e industria"/ Aziende secondario 2020
S3_SER_elec = 22180 #kWh/y consumo "Commercio e servizi" /Aziende terzario 2020

fileDir = DATA_DIRECTORY / "support_data" 
data_s123 = pd.read_csv(fileDir/"S1_S2_S3.csv" , header=0, index_col=0)

S123 = pd.DataFrame()

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

for dis_num in UST_district:
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    dis_S1_tot_elec = data_s123.loc[DISTRICT, "S1_Aziende"]*S1_AGR_elec
    dis_S2_tot_elec = data_s123.loc[DISTRICT, "S2_Aziende"]*S2_IND_elec
    dis_S3_tot_elec = data_s123.loc[DISTRICT, "S3_Aziende"]*S3_SER_elec
    
    tot_elec = pd.DataFrame(np.array([[dis_S1_tot_elec], [dis_S2_tot_elec], [dis_S3_tot_elec]]),  columns = [DISTRICT], index = [ "S1_AGR", "S2_IND", "S3_SER"]) 
    
    S123 = pd.concat([S123,tot_elec], axis = 1) #,ignore_index=True,
S123["Total"] = S123.sum(axis=1)
S123

,Bellinzona,Blenio,Leventina,Locarno,Lugano,Mendrisio,Riviera,Vallemaggia,Total
S1_AGR,10725360,7643360,7088600,12081440,15718200,11465040,2835440,4438080,71995520
S2_IND,94494750,16286900,23047500,155032850,348478200,156569350,25659550,24430350,843999450
S3_SER,86080580,7119780,11666680,118707360,362421200,123054640,12709140,8716740,730476120


### RESIDENTIAL

In [220]:
# ELECTRICITY DEMAND PER DISTRICT - Getting the number of households according to ERA (EWID) - WORKS
# 1. there are egid duplicates but each have different ewids, for the purpose of elec keep all, but need to write code that combines the ewids for each egid. For heating work with EGID since the garea is duplicated
# 2. combine with htg calc
# 3. to_csv
# 4. add a little bit of "complexity" - see if possible to seperate MFH and SFH and still get the right TI total res elect

import ast

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

TI_elec = pd.DataFrame()
district_res_elec = pd.DataFrame()
ewid_count = pd.DataFrame()
#dis_num = 2101

AVG_HH_ELEC_TI = 2741 # kWh/y USTAT 2020 = tot apparecchi/tot abitazioni ewid 
UNOCC = 0.0283 # should be around 0.0283 for TI

for dis_num in UST_district:
    communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]

    for com_num in communes:
        #print("comune: ", com_num)
        fileloc =  CANTON_CSV_DIRECTORY

        filename_gdf = "raw-gdf-"+str(com_num)+".csv"
        filename_data = "DATA_"+str(com_num)+".csv"
        
        if pathlib.Path(fileloc/filename_gdf).exists():
            data_res = pd.read_csv(fileloc/filename_gdf , header=0, index_col=0)
            #data_res = data_res_dirty.drop_duplicates(keep="first")
        elif pathlib.Path(fileloc/filename_data).exists():
            data_res = pd.read_csv(fileloc/filename_data , header=0, index_col=0)
            #data_res = data_res_dirty.drop_duplicates(keep="first")
        
        data_res = data_res.loc[data_res["gstat"] == 1004]
        data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
        data_res = data_res[REA_CODES_DESIRED_LC]


        data_res["gklas_fill"] =  data_res["gklas"]
        data_res = data_res.reset_index()

        for g in range(len(data_res)):
            if (data_res.at[g,"gkat"] == 1020):
                data_res.loc[g,"gklas_fill"] = 1110
            elif (data_res.at[g,"gkat"] == 1030):
                data_res.loc[g,"gklas_fill"] = 1122 # 63% of known 1040

        data_res["ewid"] = data_res["ewid"].fillna(value = "['no']")

        for g in range(len(data_res)):
            if (data_res.at[g,"ewid"]=="['no']"):
                if (data_res.at[g,"gklas_fill"] == 1110):
                    data_res.at[g,"ewid"] = "['1']"
                elif (data_res.at[g,"gklas_fill"] == 1121):
                    data_res.at[g,"ewid"] = "['1', '2']"
                elif (data_res.at[g,"gklas_fill"] == 1122):
                    data_res.at[g,"ewid"] = "['1', '2', '3', '4', '5', '6', '7']" # GET real median
            else:
                pass

        for g in range(len(data_res)):
            ewid_count.at[g,"ewid"] = ast.literal_eval(data_res.at[g,"ewid"])      

        data_res["ewid_len"] = ewid_count.ewid.apply(lambda x: len(x))

        district_res_elec = pd.concat([district_res_elec,data_res])
        
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    d_hh = (1-UNOCC)*(district_res_elec["ewid_len"].sum()) #
    d_elec = d_hh*AVG_HH_ELEC_TI
    d = pd.DataFrame([d_hh, d_elec], index=["hh","res_elec"], columns=[DISTRICT])

    TI_elec = pd.concat([TI_elec,d], axis=1)
    district_res_elec = district_res_elec[0:0]

TI_elec["TI"] = TI_elec.sum(axis=1)
TI_elec

,Bellinzona,Blenio,Leventina,Locarno,Lugano,Mendrisio,Riviera,Vallemaggia,TI
hh,3.288816e+04,6.942797e+03,9.396339e+03,5.741387e+04,9.589124e+04,3.033745e+04,6.240257e+03,6.692098e+03,2.458022e+05
res_elec,9.014644e+07,1.903021e+07,2.575537e+07,1.573714e+08,2.628379e+08,8.315494e+07,1.710455e+07,1.834304e+07,6.737438e+08


# TESTS

In [139]:
# Get number of households per egid
import ast

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

TI_elec = pd.DataFrame()
district_res_elec = pd.DataFrame()
ewid_count = pd.DataFrame()


AVG_HH_ELEC_TI = 2741 # kWh/y USTAT 2020 = tot apparecchi/tot abitazioni ewid 
UNOCC = 0.0283 # should be around 0.0283 for TI

dis_num = BELLINZONA_UST
com_num = 5001
#for dis_num in UST_district:
# communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]

# for com_num in communes:
    #print("comune: ", com_num)
    
fileloc =  CANTON_CSV_DIRECTORY

filename_gdf = "raw-gdf-"+str(com_num)+".csv"
filename_data = "DATA_"+str(com_num)+".csv"

if pathlib.Path(fileloc/filename_gdf).exists():
    data_res = pd.read_csv(fileloc/filename_gdf , header=0, index_col=0)
    #data_res = data_res_dirty.drop_duplicates(keep="first")
elif pathlib.Path(fileloc/filename_data).exists():
    data_res = pd.read_csv(fileloc/filename_data , header=0, index_col=0)
    #data_res = data_res_dirty.drop_duplicates(keep="first")

data_res = data_res.loc[data_res["gstat"] == 1004]
data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
data_res = data_res[REA_CODES_DESIRED_LC]

data_res["gklas_fill"] =  data_res["gklas"]
data_res = data_res.reset_index(drop=True)

for g in range(len(data_res)):
     if (np.isnan(data_res.at[g,"gklas"])):
        if (data_res.at[g,"gkat"] == 1020):
            data_res.loc[g,"gklas_fill"] = 1110
        elif (data_res.at[g,"gkat"] == 1030):
            data_res.loc[g,"gklas_fill"] = 1122

data_res["egid_duplicate"] = data_res.duplicated(subset = ["egid"], keep = False)

no_nan = data_res.dropna(subset = ["ewid"])
no_nan = no_nan.reset_index(drop=True) #,, inplace=True

for g in range(len(no_nan)):
    ewid_count.at[g,"ewid"] = ast.literal_eval(no_nan.at[g,"ewid"]) 

no_nan["ewid_len"] = ewid_count.ewid.apply(lambda x: len(x))

duplicates = no_nan.loc[no_nan["egid_duplicate"] == True]
group = duplicates.groupby("egid")["ewid_len"].sum()
group = pd.DataFrame(group)

# Replace ewid_len values in DataFrame A based on DataFrame B
no_nan['ewid_len'] = no_nan.apply(lambda row: group.loc[row['egid'], 'ewid_len'] if row['egid'] in group.index else row['ewid_len'], axis=1)

In [140]:
no_nan

,egid,strname_deinr,ggdename,ggdenr,gexpdat,gdekt,egrid,gebnr,gkode,gkodn,...,gwaerscew1,gwaerdatw1,gwaerzw2,genw2,gwaerscew2,gwaerdatw2,ewid,gklas_fill,egid_duplicate,ewid_len
0,11105003,Piazzetta 16,Arbedo-Castione,5001,22.05.2023,TI,CH468702077816,156a,2724232.580,1120786.390,...,860.0,03.05.2003,NaN,NaN,NaN,-,['1'],1110.0,False,1
1,11105004,Carrale dei Rutin 2,Arbedo-Castione,5001,22.05.2023,TI,CH268702077587,792a,2723925.473,1119284.121,...,860.0,03.05.2003,7600.0,7500.0,860.0,29.11.2001,['1'],1110.0,False,1
2,11105005,Carrale dei Rutin 4,Arbedo-Castione,5001,22.05.2023,TI,CH240702877535,790a,2723913.500,1119293.970,...,869.0,17.03.2023,7600.0,7500.0,869.0,17.03.2023,['1'],1110.0,False,1
3,11105006,Carrale dei Rutin 9,Arbedo-Castione,5001,22.05.2023,TI,CH310775028724,797a,2723887.660,1119297.610,...,860.0,03.05.2003,7600.0,7500.0,860.0,29.11.2001,"['1', '2']",1121.0,False,2
4,11105007,Via Bocarello 15,Arbedo-Castione,5001,22.05.2023,TI,CH688802078741,1097a,2723782.320,1119070.450,...,860.0,03.05.2003,7600.0,7500.0,860.0,29.11.2001,"['1', '2', '3', '4', '5', '6', '7', '8', '9', ...",1122.0,False,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,400004272,Via Campagnora 22a,Arbedo-Castione,5001,22.05.2023,TI,CH508707890259,1179b,2723903.900,1120737.930,...,869.0,13.11.2008,NaN,NaN,NaN,-,"['6', '7', '8']",1122.0,True,16
1164,400004272,Via Campagnora 20b,Arbedo-Castione,5001,22.05.2023,TI,CH508707890259,1179b,2723903.900,1120737.930,...,869.0,13.11.2008,NaN,NaN,NaN,-,"['3', '4', '5']",1122.0,True,16
1165,400004272,Via Campagnora 24a,Arbedo-Castione,5001,22.05.2023,TI,CH508707890259,1179b,2723903.900,1120737.930,...,869.0,13.11.2008,NaN,NaN,NaN,-,"['12', '13', '14']",1122.0,True,16
1166,190220368,Via Giardino 25,Arbedo-Castione,5001,22.05.2023,TI,CH768702078504,1701d,2724078.260,1119829.080,...,869.0,28.12.2009,NaN,NaN,NaN,-,"['1', '2']",1121.0,False,2


In [148]:
abi_num = no_nan[["gklas_fill","ewid_len","gdekt"]]

In [150]:
print(len(abi_num))
print(abi_num.isna().sum())

1168
gklas_fill    0
ewid_len      0
gdekt         0
dtype: int64


In [157]:
abi_num = no_nan[["gklas_fill","ewid_len","gdekt"]]
abi_num = abi_num.groupby(["gklas_fill","ewid_len"]).count()
abi_num = abi_num.rename(columns={"gdekt":"count"})
abi_num.rename_axis(index={"gklas_fill":"gklas","ewid_len":"number of households"}, inplace = True)
abi_num

count
gklas  number of households       
1110.0 1                       735
       2                         5
1121.0 2                       190
       3                         2
1122.0 3                        55
       4                        23
       5                        11
       6                        37
       7                         8
       8                        26
       9                        15
       10                        2
       11                        1
       12                       13
       13                        1
       14                        2
       15                       16
       16                        9
       17                        1
       18                        2
       20                        3
       21                        2
       24                        1
       29                        3
       31                        2
       51                        3

In [130]:
import pandas as pd

# Sample DataFrames
data_A = {
    'egid': [1, 2, 3, 1, 2],
    'ewid': [10, 20, 30, 40, 50],
    'ewid_len': [100, 200, 300, 400, 500]
}
df_A = pd.DataFrame(data_A)

data_B = {
    'egid': [1, 2],
    'ewid_len': [999, 888]
}
df_B = pd.DataFrame(data_B)
df_B = df_B.set_index('egid')

# Display the original DataFrames
print("Original DataFrame A:")
print(df_A)
print("\nOriginal DataFrame B:")
print(df_B)

# Replace ewid_len values in DataFrame A based on DataFrame B
df_A['ewid_len'] = df_A.apply(lambda row: df_B.loc[row['egid'], 'ewid_len'] if row['egid'] in df_B.index else row['ewid_len'], axis=1)

# Display the modified DataFrame A
print("\nModified DataFrame A:")
print(df_A)

Original DataFrame A:
   egid  ewid  ewid_len
0     1    10       100
1     2    20       200
2     3    30       300
3     1    40       400
4     2    50       500

Original DataFrame B:
      ewid_len
egid          
1          999
2          888

Modified DataFrame A:
   egid  ewid  ewid_len
0     1    10       999
1     2    20       888
2     3    30       300
3     1    40       999
4     2    50       888


In [142]:
fileDir = DATA_DIRECTORY / "support_data" 
data_s123 = pd.read_csv(fileDir/"S1_S2_S3.csv" , header=0, index_col=0)
data_s123

,Totale__Imprese,S1_Imprese,S2_Imprese,S3_Imprese,Totale_Aziende,S1_Aziende,S2_Aziende,S3_Aziende
Mendrisio,6126,176,980,4970,6753,186,1019,5548
Lugano,17354,247,2203,14904,18863,255,2268,16340
Locarno,5885,189,970,4726,6557,196,1009,5352
Vallemaggia,566,68,153,345,624,72,159,393
Bellinzona,3835,168,582,3085,4670,174,615,3881
Riviera,675,47,155,473,786,46,167,573
Blenio,516,124,108,284,551,124,106,321
Leventina,680,111,126,443,791,115,150,526


### S1 -AGRICULTURE, S2 -INDUSTRY, S3 -SERVICES

In [15]:
# LOOP THROUGH ALL DISTRICT
#for dis_num in UST_district:

from operator import itemgetter

DICT = {"SFH":1110.0, "DFH":1121.0, "MFH":1122.0, "HABITAT_COMMUNAUTAIRE":1130.0, "HOTEL":1211.0, "HEBERGEMENT":1212.0, "OFFICE":1220.0, "COMMERCIAL":1230.0, "RESTO_BAR":1231.0, "TRANSP_STATIONS":1241.0, "GARAGE": 1242.0, 
        "INDUSTRIAL":1251.0, "RESERVOIRS":1252.0, "CULTURAL":1261.0, "MUSEUM_LIBRARY":1262.0, "ACADEMIC":1263.0, "HOSPITAL":1264.0, "SPORTS":1265.0, "AGRICULTURE":1271.0, "RELIGIOUS":1272.0, "ANCIENT":1273.0, "OTHER_PUBLIC":1274.0, "OUTSIDE": 1275.0,
        "ANIMALS":1276.0, "GREENHOUSE":1277.0, "AGRI_STORAGE":1278.0}

RES_code = itemgetter("SFH", "DFH", "MFH")(DICT)
AGR_code = itemgetter("AGRICULTURE")(DICT) #, "ANIMALS", "GREENHOUSE", "AGRI_STORAGE"
IND_code = itemgetter("INDUSTRIAL")(DICT)
SER_code = itemgetter("HABITAT_COMMUNAUTAIRE","HOTEL", "HEBERGEMENT", "OFFICE", "COMMERCIAL", "RESTO_BAR", "TRANSP_STATIONS","CULTURAL", "MUSEUM_LIBRARY", "ACADEMIC", "HOSPITAL", "SPORTS")(DICT)
OTHER_code = itemgetter("GARAGE","RESERVOIRS", "ANCIENT", "OUTSIDE", "RELIGIOUS","OTHER_PUBLIC")(DICT)

# SINGLE DISTRICT
dis_num = LUGANO_UST
DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
dis_filename = DISTRICT + ".csv"
dis_csv = CANTON_CSV_DIRECTORY / dis_filename
dis_rea = pd.read_csv(dis_csv, sep= ";", header=0, index_col=0).set_index("EGID")
dis_rea = dis_rea.drop(["index.1"], axis=1)

dr = dis_rea.drop(["GABBJ","GSTAT", "GBAUJ", "GBAUP","GWAERZW1","GENW1","GWAERDATW1", "GWAERDATH1", "GENH1", "GWAERZH1","GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1) #
dr = dr.reset_index()
#dr.dtypes

RES = dr.loc[dr["GKLAS_fill"].isin(RES_code)]
AGR = dr.loc[dr["GKLAS_fill"] == (AGR_code)]
IND = dr.loc[dr["GKLAS_fill"] == (IND_code)]
SER = dr.loc[dr["GKLAS_fill"].isin(SER_code)]
OTHER = dr.loc[dr["GKLAS_fill"].isin(OTHER_code)]

print(DISTRICT, "RES:", RES["EGID"].count() , "AGR:", AGR["EGID"].count(), "IND:", IND["EGID"].count() , "SER:", SER["EGID"].count(), "OTHER:", OTHER["EGID"].count())

Lugano RES: 37101 AGR: 5758 IND: 547 SER: 2231 OTHER: 9221


### RESIDENTIAL

In [131]:
# PER COMUNE -  Getting the number of households according to ERA (EWID) -> matches pretty well with this layer: https://map.geo.admin.ch/?layers=ch.are.wohnungsinventar-zweitwohnungsanteil
import ast

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]


#for com_num in communes:
com_num = 5269    
print("comune: ", com_num)
fileloc =  CANTON_CSV_DIRECTORY
# reading comune raw gdf file
filename = "raw-gdf-"+str(com_num)+".csv"
data_res = pd.read_csv(fileloc/filename , header=0, index_col=0)
data_res = data_res.loc[data_res["gstat"] == 1004]
data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
data_res = data_res[REA_CODES_DESIRED_LC]


data_res["gklas_fill"] =  data_res["gklas"]
data_res = data_res.reset_index()

for g in range(len(data_res)):
    if (data_res.at[g,"gkat"] == 1020):
        data_res.loc[g,"gklas_fill"] = 1110
    elif (data_res.at[g,"gkat"] == 1030):
        data_res.loc[g,"gklas_fill"] = 1122 # 63% of known 1040
        
data_res["ewid"] = data_res["ewid"].fillna(value = "['no']")

for g in range(len(data_res)):
    if (data_res.at[g,"ewid"]=="['no']"):
        if (data_res.at[g,"gklas_fill"] == 1110):
            data_res.at[g,"ewid"] = "['1']"
        elif (data_res.at[g,"gklas_fill"] == 1121):
            data_res.at[g,"ewid"] = "['1', '2']"
        elif (data_res.at[g,"gklas_fill"] == 1122):
            data_res.at[g,"ewid"] = "['1', '2', '3', '4', '5', '6', '7']" # GET real median
    else:
        pass

for g in range(len(data_res)):
    data_res.at[g,"ewid"] = ast.literal_eval(data_res.at[g,"ewid"])      

data_res["ewid_len"] = data_res.ewid.apply(lambda x: len(x))

test = data_res[["gklas_fill","ewid","ewid_len"]]

print(test["ewid_len"].sum())


comune:  5269
1493


In [33]:
## PER COMUNE
REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]
b = district_res_elec.loc[district_res_elec["ggdenr"] == 5009]# GGDENR Numéro OFS de la commune
b = b[REA_CODES_DESIRED_LC]
b = b.reset_index()
b.isna().sum()

index              0
egid               0
strname_deinr      0
ggdename           0
ggdenr             0
gexpdat            0
gdekt              0
egrid            204
gebnr              0
gkode              0
gkodn              0
gksce              0
gstat              0
gkat               0
gklas             65
gbauj            318
gbaup             86
gabbj            486
garea              0
gastw             54
gazzi            486
gebf             486
gwaerzh1         226
genh1            226
gwaersceh1       226
gwaerdath1         0
gwaerzh2         458
genh2            458
gwaersceh2       458
gwaerdath2         0
gwaerzw1         226
genw1            226
gwaerscew1       226
gwaerdatw1         0
gwaerzw2         366
genw2            366
gwaerscew2       366
gwaerdatw2         0
ewid             233
dtype: int64

In [36]:
# AGE GROUPBY - COUNT
b = district_res_elec.loc[district_res_elec["ggdenr"] == 5009]# GGDENR Numéro OFS de la commune

GKAT_group = b[["gkat", "gklas", "ewid"]]
GKAT_group = GKAT_group.groupby(["gkat", "gklas"]).count()
GKAT_group

ewid
gkat gklas       
1020 1110.0   205
     1121.0    34
     1122.0     7
1030 1110.0     1
     1121.0     1
1040 1211.0     1
1060 1220.0     0
     1242.0     0
     1251.0     0
     1252.0     0
     1263.0     0
     1271.0     1
     1272.0     0
     1274.0     0

In [17]:
# (1/3) Calculating number of households and ppl, and elec consumption - DOESN'tGIVE REASONABLE VALUES
for dis_num in UST_district:
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    dis_filename = DISTRICT + ".csv"
    dis_csv = CANTON_CSV_DIRECTORY / dis_filename
    dis_rea = pd.read_csv(dis_csv, sep= ";", header=0, index_col=0).set_index("EGID")
    dis_rea = dis_rea.drop(["index.1"], axis=1)
    dr = dis_rea.drop(["GABBJ","GSTAT", "GBAUJ", "GBAUP","GWAERZW1","GENW1","GWAERDATW1", "GWAERDATH1", "GENH1", "GWAERZH1","GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1) #
    dr = dr.reset_index()
    
    dr["ppl"] = 2.4
    dr["elec"] = SFH_4pp_elec
    dr["num_hh"] = 1

for g in range(len(dr)):
    
    ### RESIDENTIAL
    if dr.at[g,"GKLAS_fill"] == 1110:
        dr.at[g,"ppl"] = min(0.93*SFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/AVG_M2_PP))
        dr.at[g,"elec"] = SFH_4pp_elec + (dr.at[g,"ppl"]-4)*SFH_1pp_elec_diff
        
    elif dr.at[g,"GKLAS_fill"] == 1121:
        dr.at[g,"num_hh"] = 2
        dr.at[g,"ppl"] = dr.at[g,"num_hh"]*(min(1.1*MFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/dr.at[g,"num_hh"]/AVG_M2_PP)))
        dr.at[g,"elec"] = SFH_4pp_elec + (dr.at[g,"ppl"]-4)*SFH_1pp_elec_diff
    
    elif dr.at[g,"GKLAS_fill"] == 1122:
#        if (dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]) <SUPERFICIE_MEDIA:
#            dr.at[g,"num_hh"] = dr.at[g,"GASTW_fill"]
#        else:
        dr.at[g,"num_hh"] = 0.6*(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"])/(1.5*SUPERFICIE_MEDIA)
        dr.at[g,"ppl"] = dr.at[g,"num_hh"]*min(1.1*MFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/dr.at[g,"num_hh"]/AVG_M2_PP))
        dr.at[g,"elec"] = MFH_2pp_elec + (dr.at[g,"ppl"]-2)*MFH_1pp_elec_diff    
    
    ### SERVICES
    else:
        dr.at[g,"ppl"] = 0
        dr.at[g,"elec"] = 0

In [18]:
# (2/3) Calculating number of households and ppl, and elec consumption - DOESN'tGIVE REASONABLE VALUES
dr_SFH = dr.loc[dr["GKLAS_fill"] == 1110]
print("SFH_median: ",round(dr_SFH["ppl"].median(),1), "SFH_mean: ",round(dr_SFH["ppl"].mean(),1))
        
dr_DFH = dr.loc[dr["GKLAS_fill"] == 1121]
print("DFH_median: ", round(dr_DFH["ppl"].median()/dr_DFH["num_hh"].median(),1), "DFH_mean: ",round(dr_DFH["ppl"].mean()/dr_DFH["num_hh"].median(),1))

dr_MFH = dr.loc[dr["GKLAS_fill"] == 1122]
print("MFH_median: ",round(dr_MFH["ppl"].median()/dr_MFH["num_hh"].median(),1), "MFH_mean: ",round(dr_MFH["ppl"].mean()/dr_MFH["num_hh"].median(),1))

SFH_median:  2.3 SFH_mean:  2.0
DFH_median:  2.3 DFH_mean:  2.1
MFH_median:  2.3 MFH_mean:  3.1


In [19]:
# (3/3) Calculating number of households and ppl, and elec consumption - DOESN'tGIVE REASONABLE VALUES
dr_hh = dr["num_hh"].sum()
dr_ppl = dr["ppl"].sum()
dr_elec = dr["elec"].sum()
print( DISTRICT," hh:",dr_hh, " ppl:",dr_ppl, " elec kWh:", dr_elec)

Vallemaggia  hh: 12358.69529652352  ppl: 13111.829279572177  elec kWh: 17332612.34920523
